# Construccion de Carteras Optimas

## 3 Estrategias de Inversion

In [1]:
import sys
sys.path.insert(0, '../codigo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from construccion_carteras import CarteraOptimaMercado, CarteraAgresivaTech, CarteraGrowthMomentum
from analisis_numero_optimo import AnalisisNumeroOptimo
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print('[OK] Librerias importadas')

[OK] Librerias importadas


In [2]:
datos_retornos = pd.read_excel('../../data/prod_long_sharpe_u60_20260125_v1_train_dataset.csv', sheet_name='Sheet1')
datos_retornos.columns = [f'Activo_{i}' for i in range(1, 61)]

betas_df = pd.read_csv('../betas_resultados.csv')

caracteristicas_df = pd.read_excel('../../data/prod_long_sharpe_u60_20260125_v1_train_dataset.csv', sheet_name='Hoja2')

print(f'[OK] Datos cargados: {datos_retornos.shape}')

[OK] Datos cargados: (1758, 60)


In [3]:
analizador = AnalisisNumeroOptimo(datos_retornos)
df_frontera = analizador.simular_frontera_diversificacion(
    n_valores=[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 40, 50, 60],
    n_simulaciones=100
)

n_optimo = analizador.detectar_n_optimo(umbral_reduccion=2.0)
print(f'[OK] N optimo detectado: {n_optimo}')


[OK] Análisis de número óptimo inicializado
  Activos disponibles: 60
  Observaciones: 1758

SIMULACIÓN: FRONTERA DE DIVERSIFICACIÓN
Valores de N a probar: [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20, 25, 30, 40, 50, 60]
Simulaciones por N: 100
Total de simulaciones: 1700
--------------------------------------------------------------------------------

Procesando N=2...

Procesando N=3...

Procesando N=4...

Procesando N=5...

Procesando N=6...

Procesando N=7...

Procesando N=8...

Procesando N=9...

Procesando N=10...

Procesando N=12...

Procesando N=15...

Procesando N=20...

Procesando N=25...

Procesando N=30...

Procesando N=40...

Procesando N=50...

Procesando N=60...

TABLA RESUMEN: FRONTERA DE DIVERSIFICACIÓN
  N |  Vol(%) |   ±Std |  Esp(%) |  Sis(%) |  Reduc
--------------------------------------------------------------------------------
  2 |    1.61 |   0.32 |   86.48 |   41.17 |    nan%
  3 |    1.37 |   0.22 |   83.23 |   56.03 |  15.16%
  4 |    1.28 |   0.18 |   77.31 |

## Cartera 1: Optima de Mercado

In [4]:
cartera1 = CarteraOptimaMercado(datos_retornos, betas_df, caracteristicas_df, n_optimo)
activos_optimos = cartera1.seleccionar_activos()

print(f'[OK] Activos seleccionados para cartera optima:')
print(activos_optimos[['Activo', 'Sharpe', 'Volatilidad_Anualizada', 'Beta', 'Sector']].to_string(index=False))

[OK] Activos seleccionados para cartera optima:
 Activo   Sharpe  Volatilidad_Anualizada     Beta                         Sector
     51 1.411636                0.193692 0.371391 Residential & Commercial REITs
     56 1.322656                0.183033 0.305494 Residential & Commercial REITs
     47 1.185120                0.174693 0.318452 Residential & Commercial REITs
     41 1.096431                0.191392 0.404310 Residential & Commercial REITs
     50 0.865891                0.217200 0.289355      Electric Utilities & IPPs


In [5]:
pesos_cartera1 = cartera1.optimizar_pesos()

activos_con_peso = [(i+1, pesos_cartera1[i]) for i in range(60) if pesos_cartera1[i] > 0.001]
activos_con_peso.sort(key=lambda x: x[1], reverse=True)

print(f'\nCartera Optima de Mercado - Vector de Pesos:')
print(f'==========================================')
for activo, peso in activos_con_peso:
    print(f'Activo {activo:2d}: {peso:.6f} ({peso*100:.2f}%)')
print(f'Total: {sum([p for _, p in activos_con_peso]):.6f}')

print(f'\nVector de pesos completo (60 activos):')
print(pesos_cartera1)


Cartera Optima de Mercado - Vector de Pesos:
Activo 51: 0.350527 (35.05%)
Activo 56: 0.259366 (25.94%)
Activo 50: 0.211806 (21.18%)
Activo 47: 0.102999 (10.30%)
Activo 41: 0.075303 (7.53%)
Total: 1.000000

Vector de pesos completo (60 activos):
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.07530267 0.
 0.         0.         0.         0.         0.10299924 0.
 0.         0.21180576 0.35052662 0.         0.         0.
 0.         0.25936571 0.         0.         0.         0.        ]


## Cartera 2: Agresiva Tech

In [6]:
cartera2 = CarteraAgresivaTech(datos_retornos, betas_df, caracteristicas_df)
activos_tech = cartera2.seleccionar_activos_tech()

print(f'[OK] Activos Tech seleccionados:')
print(activos_tech[['Activo', 'Beta', 'Volatilidad_Anualizada', 'Sector']].to_string(index=False))

[OK] Activos Tech seleccionados:
 Activo     Beta  Volatilidad_Anualizada                 Sector
     38 1.238451                0.354014 Software & IT Services
     42 1.796005                0.537733 Software & IT Services


In [7]:
pesos_cartera2 = cartera2.optimizar_pesos()

activos_con_peso_2 = [(i+1, pesos_cartera2[i]) for i in range(60) if pesos_cartera2[i] > 0.001]
activos_con_peso_2.sort(key=lambda x: x[1], reverse=True)

print(f'\nCartera Agresiva Tech - Vector de Pesos:')
print(f'=========================================')
for activo, peso in activos_con_peso_2:
    print(f'Activo {activo:2d}: {peso:.6f} ({peso*100:.2f}%)')
print(f'Total: {sum([p for _, p in activos_con_peso_2]):.6f}')

print(f'\nVector de pesos completo (60 activos):')
print(pesos_cartera2)


Cartera Agresiva Tech - Vector de Pesos:
Activo 42: 1.000000 (100.00%)
Total: 1.000000

Vector de pesos completo (60 activos):
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 8.32667268e-17 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.

## Cartera 3: Growth + Momentum

In [8]:
cartera3 = CarteraGrowthMomentum(datos_retornos, betas_df, caracteristicas_df)
activos_growth = cartera3.seleccionar_activos()

print(f'[OK] Activos Growth+Momentum seleccionados:')
print(activos_growth[['Activo', 'Retorno_Anualizado', 'Momentum', 'Beta', 'Sector']].to_string(index=False))

[OK] Activos Growth+Momentum seleccionados:
 Activo  Retorno_Anualizado  Momentum     Beta                 Sector
      9            0.065798 -0.714570 1.532317        Pharmaceuticals
     38           -0.074836  4.601624 1.238451 Software & IT Services
     33            0.118259 -1.961766 1.086463    Specialty Retailers
     42           -0.077984 -1.707325 1.796005 Software & IT Services
     27           -0.017557 -1.855147 1.048143 Software & IT Services
     26           -0.060918 -2.749199 1.144704    Specialty Retailers


In [9]:
pesos_cartera3 = cartera3.optimizar_pesos()

activos_con_peso_3 = [(i+1, pesos_cartera3[i]) for i in range(60) if pesos_cartera3[i] > 0.001]
activos_con_peso_3.sort(key=lambda x: x[1], reverse=True)

print(f'\nCartera Growth+Momentum - Vector de Pesos:')
print(f'==========================================')
for activo, peso in activos_con_peso_3:
    print(f'Activo {activo:2d}: {peso:.6f} ({peso*100:.2f}%)')
print(f'Total: {sum([p for _, p in activos_con_peso_3]):.6f}')

print(f'\nVector de pesos completo (60 activos):')
print(pesos_cartera3)


Cartera Growth+Momentum - Vector de Pesos:
Activo 33: 0.930457 (93.05%)
Activo  9: 0.069543 (6.95%)
Total: 1.000000

Vector de pesos completo (60 activos):
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.95428290e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.44383040e-16 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 9.30457171e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00

## Resumen de Carteras

In [ ]:
retornos_c1 = np.sum(pesos_cartera1 * datos_retornos.mean() * 252)
vol_c1 = np.sqrt(np.dot(pesos_cartera1, np.dot(datos_retornos.cov() * 252, pesos_cartera1)))
sharpe_c1 = retornos_c1 / vol_c1 if vol_c1 > 0 else 0

retornos_c2 = np.sum(pesos_cartera2 * datos_retornos.mean() * 252)
vol_c2 = np.sqrt(np.dot(pesos_cartera2, np.dot(datos_retornos.cov() * 252, pesos_cartera2)))
sharpe_c2 = retornos_c2 / vol_c2 if vol_c2 > 0 else 0

retornos_c3 = np.sum(pesos_cartera3 * datos_retornos.mean() * 252)
vol_c3 = np.sqrt(np.dot(pesos_cartera3, np.dot(datos_retornos.cov() * 252, pesos_cartera3)))
sharpe_c3 = retornos_c3 / vol_c3 if vol_c3 > 0 else 0

print(f'\nRESUMEN DE CARTERAS')
print(f'{'='*50}\n')
print(f'Cartera 1: Optima de Mercado')
print(f'  Retorno anualizado: {retornos_c1:.4f}')
print(f'  Volatilidad: {vol_c1:.4f}')
print(f'  Sharpe Ratio: {sharpe_c1:.4f}')
print(f'  Activos: {len(activos_con_peso)}')

print(f'\nCartera 2: Agresiva Tech')
print(f'  Retorno anualizado: {retornos_c2:.4f}')
print(f'  Volatilidad: {vol_c2:.4f}')
print(f'  Sharpe Ratio: {sharpe_c2:.4f}')
print(f'  Activos: {len(activos_con_peso_2)}')

print(f'\nCartera 3: Growth + Momentum')
print(f'  Retorno anualizado: {retornos_c3:.4f}')
print(f'  Volatilidad: {vol_c3:.4f}')
print(f'  Sharpe Ratio: {sharpe_c3:.4f}')
print(f'  Activos: {len(activos_con_peso_3)}')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

pesos_c1_plot = [p for _, p in activos_con_peso]
labels_c1 = [f"A{a}" for a, _ in activos_con_peso]
axes[0].bar(range(len(pesos_c1_plot)), pesos_c1_plot)
axes[0].set_title('Cartera Optima - Pesos', fontweight='bold')
axes[0].set_xticks(range(len(labels_c1)))
axes[0].set_xticklabels(labels_c1, rotation=45)
axes[0].set_ylabel('Peso')

pesos_c2_plot = [p for _, p in activos_con_peso_2]
labels_c2 = [f"A{a}" for a, _ in activos_con_peso_2]
axes[1].bar(range(len(pesos_c2_plot)), pesos_c2_plot, color='orange')
axes[1].set_title('Cartera Tech - Pesos', fontweight='bold')
axes[1].set_xticks(range(len(labels_c2)))
axes[1].set_xticklabels(labels_c2, rotation=45)
axes[1].set_ylabel('Peso')

pesos_c3_plot = [p for _, p in activos_con_peso_3]
labels_c3 = [f"A{a}" for a, _ in activos_con_peso_3]
axes[2].bar(range(len(pesos_c3_plot)), pesos_c3_plot, color='green')
axes[2].set_title('Cartera Growth+Momentum - Pesos', fontweight='bold')
axes[2].set_xticks(range(len(labels_c3)))
axes[2].set_xticklabels(labels_c3, rotation=45)
axes[2].set_ylabel('Peso')

plt.tight_layout()
plt.savefig('carteras_pesos.png', dpi=300, bbox_inches='tight')
plt.show()

print('[OK] Grafico guardado: carteras_pesos.png')

In [ ]:
pd.DataFrame({
    'Cartera_Optima_Mercado': pesos_cartera1,
    'Cartera_Agresiva_Tech': pesos_cartera2,
    'Cartera_Growth_Momentum': pesos_cartera3
}).to_csv('../vectores_pesos_carteras.csv', index=False)

print('[OK] Vectores de pesos exportados: vectores_pesos_carteras.csv')